# DSFB Project - Part 2

### 1. Data labelling




##### 1.1 imports and paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import zipfile

from PIL import Image
from io import BytesIO


from tqdm import tqdm
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import BatchNormalization, Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Paths to files
# Variables to edit according to your paths to the project files
# Here, the image folder have to be zipped to be read properly by the algorithm

# Path to the project
PATH_TO_PROJECT_DSB = '/content/drive/My Drive/Project_DSB'

# Paths to the files and folders
PATH_TO_LABELS_FILE = PATH_TO_PROJECT_DSB + '/train_data/Labels.txt'
PATH_TO_IMAGES_FILE = PATH_TO_PROJECT_DSB + '/train_data/Character_Images.zip'
PATH_TO_MODELS = PATH_TO_PROJECT_DSB + '/models'

##### 1.2 Treat labels and create the Y vector

In [ ]:
# Load labels DataFrame
labels_df = pd.read_csv(PATH_TO_LABELS_FILE, sep=',', header=None, names=['Id', 'Label'])
labels_df['Id'] = labels_df['Id'].apply(lambda x: int(x[-5:]))
labels_df['Label'] = labels_df['Label'].apply(lambda x: str(x)[1:]).astype(str)
y = labels_df['Label'].to_numpy()
labels_df.head()

,Id,Label
0,0,5
1,1,5
2,2,8
3,3,5
4,4,9


In [ ]:
# Display unique labels
print('The labels are : ', labels_df['Label'].unique())

The labels are :  ['5' '8' '9' '1' '0' '7' '2' 'X' '3' '6' 'C' '4' 'E']


In [ ]:

# The to_categorical function must take integers as input
# Hence, we map each string (i.e. C, E and X) to a unique integer
def map_labels_to_int(x):
  try:
    output = int(x)
  except:
    if x=='C':
      output = 10
    elif x=='E':
      output = 11
    elif x=='X':
      output = 12
    else:
      print('Class problem : ', x)
  return output

Y = np.array([map_labels_to_int(yi) for yi in y])

In [ ]:
print('The new unique labels, in a suitable format for a ML algorithm, are :', np.unique(Y))

The new unique labels, in a suitable format for a ML algorithm, are : [ 0  1  2  3  4  5  6  7  8  9 10 11 12]


#### 1.3 Load images into the right format

In [ ]:
# Images data are contained in a zipped folder.
# We then must create a function to load them into multidimentional arrays
# The function :
def extract_images_from_zip(zip_path):
    image_names = []
    image_vectors = []

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for i, file_info in enumerate(tqdm(zip_ref.infolist())):
            if file_info.is_dir():
                continue

            image_name = file_info.filename
            image_names.append(int(image_name[-4-5:-4]))

            with zip_ref.open(file_info) as file:
                image_bytes = file.read()
                image = Image.open(BytesIO(image_bytes))
                image_array = np.array(image)
                image_vectors.append(image_array)

    return image_names, np.array(image_vectors)

image_names, image_vectors = extract_images_from_zip(PATH_TO_IMAGES_FILE)

print("\nNumber of Image:")
print(len(image_names))

print("\nImages Vector shape (nb of images, height, width, colors dimensions) :")
print(image_vectors.shape)

100%|██████████| 74944/74944 [00:24<00:00, 3017.46it/s]



Number of Image:
74943

Images Vector shape (nb of images, height, width, colors dimensions) :
(74943, 64, 32, 3)


#### 1.4 Prepare data format to be processed by a ML model

In [ ]:
# We split into train/test dataset
# We choose a train size of 60000, wich gives a test size of roughly 15000
train_size = 60000
X_train, X_test, y_train, y_test = train_test_split(image_vectors, Y, train_size = train_size, random_state=23)

# change integers to 32-bit floating point numbers
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalize each value for each pixel for the entire vector for each input
X_train /= 255
X_test /= 255

# one-hot format classes
nb_classes = labels_df['Label'].nunique() # number of unique labels

Y_train = to_categorical(y_train, nb_classes,)
Y_test = to_categorical(y_test, nb_classes, )

# delete unuseful vectors to save memory
del Y, image_vectors, y

### 2. Training of the Neural Network

#### 2.1 Define the model architecture

We inspired ourselves form the model architecture of the exercices, but we changed the kernel size to make it suitable for our images shape (64,32). We thought that a 4x4 kernel would work well.

In [ ]:
# Model architecture inspired from the exercises lecture :
# "Introduction to Deep Learning with Keras and TensorFlow" of this course

# Model instantiation
model = Sequential()                                 # Linear stacking of layers

# Convolution Layer 1
model.add(Conv2D(64, (3, 3), input_shape=(64, 32, 3))) # 64 different 3x3 kernels -- so 32 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
convLayer01 = Activation('relu')                     # activation
model.add(convLayer01)

# Convolution Layer 2
model.add(Conv2D(64, (3, 3)))                        # 64 different 3x3 kernels -- so 32 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
model.add(Activation('relu'))                        # activation
convLayer02 = MaxPooling2D(pool_size=(2,2))          # Pool the max values over a 2x2 kernel
model.add(convLayer02)

# Convolution Layer 3
model.add(Conv2D(128,(3, 3)))                         # 128 different 3x3 kernels -- so 64 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
convLayer03 = Activation('relu')                     # activation
model.add(convLayer03)

# Convolution Layer 4
model.add(Conv2D(128, (3, 3)))                        # 128 different 3x3 kernels -- so 64 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
model.add(Activation('relu'))                        # activation
convLayer04 = MaxPooling2D(pool_size=(2,2))          # Pool the max values over a 2x2 kernel
model.add(convLayer04)
model.add(Flatten())                                 # Flatten final 4x4x64 output matrix into a 1024-length vector

# Fully Connected Layer 5
model.add(Dense(512))                                # 512 FCN nodes
model.add(BatchNormalization())                      # normalization
model.add(Activation('relu'))                        # activation

# Fully Connected Layer 6
model.add(Dropout(0.2))                              # 20% dropout of randomly selected nodes
model.add(Dense(nb_classes))                         # final nb_classes FCN nodes
model.add(Activation('softmax'))                     # Softmax output the probability that each label is actually the label of the image

# We compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# We display a summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 62, 30, 64)        1792      
                                                                 
 batch_normalization_5 (Bat  (None, 62, 30, 64)        256       
 chNormalization)                                                
                                                                 
 activation_6 (Activation)   (None, 62, 30, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 28, 64)        36928     
                                                                 
 batch_normalization_6 (Bat  (None, 60, 28, 64)        256       
 chNormalization)                                                
                                                                 
 activation_7 (Activation)   (None, 60, 28, 64)       

#### 2.2 Data augmentation process

In [ ]:
# We use Keras built-in feature to do automatic augmentation : ImageDataGenerator

gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

test_gen = ImageDataGenerator()

#### 2.3 We define the data loader in order to keep memory usage optimal

In [ ]:
# we define the number of epochs to train our data on and the batch size
nb_epochs = 15
batch_size = 128

train_generator = gen.flow(X_train, Y_train, batch_size=batch_size)
test_generator = test_gen.flow(X_test, Y_test, batch_size=batch_size)

#### 2.4 We train the model

In [ ]:
# We train our model
model.fit_generator(train_generator, steps_per_epoch=len(Y_train)//batch_size, epochs=nb_epochs, verbose=1,
                    validation_data=test_generator, validation_steps=len(Y_test)//batch_size)

<ipython-input-14-c34b93c814e0>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=len(Y_train)//batch_size, epochs=optimal_nb_epochs, verbose=1,


Epoch 1/15
468/468 [==============================] - 82s 143ms/step - loss: 0.0918 - accuracy: 0.9747 - val_loss: 0.1230 - val_accuracy: 0.9807
Epoch 2/15
468/468 [==============================] - 67s 144ms/step - loss: 0.0211 - accuracy: 0.9941 - val_loss: 0.0315 - val_accuracy: 0.9894
Epoch 3/15
468/468 [==============================] - 66s 141ms/step - loss: 0.0147 - accuracy: 0.9955 - val_loss: 0.0035 - val_accuracy: 0.9991
Epoch 4/15
468/468 [==============================] - 65s 139ms/step - loss: 0.0111 - accuracy: 0.9967 - val_loss: 0.0040 - val_accuracy: 0.9989
Epoch 5/15
468/468 [==============================] - 65s 138ms/step - loss: 0.0096 - accuracy: 0.9973 - val_loss: 0.0270 - val_accuracy: 0.9918
Epoch 6/15
468/468 [==============================] - 65s 140ms/step - loss: 0.0087 - accuracy: 0.9973 - val_loss: 0.0032 - val_accuracy: 0.9990
Epoch 7/15
468/468 [==============================] - 64s 137ms/step - loss: 0.0069 - accuracy: 0.9980 - val_loss: 0.0043 - val_ac

#### 2.5 We display the results of our model

In [ ]:
# We display the train and test performances of our model
train_score = model.evaluate(X_train, Y_train)
test_score = model.evaluate(X_test, Y_test)

print('\nTrain score:', train_score[0])
print('Train accuracy: {:.5} %'.format(100*train_score[1]))

print('\nTest score:', test_score[0])
print('Test accuracy: {:.5} %'.format(100*test_score[1]))

467/467 [==============================] - 3s 6ms/step - loss: 0.0033 - accuracy: 0.9993

Train score: 0.0005997331463731825
Train accuracy: 99.987 %

Test score: 0.003332382533699274
Test accuracy: 99.933 %


#### 2.6 We save our model weights

In [ ]:
# Save the wights of the model we trained
model_name = '/model_jehan.keras'
model.save(PATH_TO_MODELS + model_name)

In [ ]:
# Load the model we trained
# model = keras.models.load_model(PATH_TO_MODELS + model_name)